In [1]:
import pandas as pd
import glob

In [16]:
pathways = []

for name in glob.glob("F:\\pathways\\*\\*\\*\\*", recursive = True):
        pathways.append(name.split("\\"))

In [17]:
pathways[0]

['F:', 'pathways', '0006NR23203', 'I', '0_0.237', '0.003_Roadway.jpg']

In [18]:
pathways_df = pd.DataFrame(pathways, columns = ["drive", "source", "route", "type", "mile", "image"])

In [21]:
pathways_df.drop("drive", axis = 1, inplace = True)

In [44]:
pathways_df.shape

(1945840, 5)

In [48]:
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS

def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

def dms_to_decimal(degrees, minutes, seconds, direction):
    """Converts degrees, minutes, seconds to decimal degrees."""
    decimal_degrees = degrees + minutes/60 + seconds/3600
    if direction in ['S', 'W']:
        decimal_degrees *= -1
    return decimal_degrees

def get_coordinates(geotags):
    lat = dms_to_decimal(*latitude_dms, latitude_ref)
    lon = dms_to_decimal(*longitude_dms, longitude_ref)

    return (float(round(lat, 5)), float(round(lon, 5)))

pathways = []

for name in glob.glob("F:\\pathways\\*\\*\\*\\*", recursive = True):
        
    image = Image.open(name)
    exif = image._getexif()
    geotags = get_geotagging(exif)

    latitude_ref = geotags['GPSLatitudeRef']
    latitude_dms = geotags['GPSLatitude']
    longitude_ref = geotags['GPSLongitudeRef']
    longitude_dms = geotags['GPSLongitude']
    GPSTimeStamp = geotags['GPSTimeStamp']
    GPSDateStamp = geotags['GPSDateStamp']
    
    pathways.append([name.split("\\"), get_coordinates(geotags), GPSTimeStamp, GPSDateStamp])

In [49]:
pathways[0]

[['F:', 'pathways', '0006NR23203', 'I', '0_0.237', '0.003_Roadway.jpg'],
 (39.68843, -110.85625),
 (18.0, 34.0, 38.571),
 '2024:08:06']

In [65]:
pathways_image_df = pd.DataFrame(pathways, columns = ['identifier_info','coordinates','GPSTimeStamp', 'GPSDateStamp'])

In [66]:
pathways_image_df[['drive','mode','route','type','mile_marker', 'image']] = pd.DataFrame(pathways_image_df['identifier_info'].tolist(), index= pathways_image_df.index)

In [67]:
pathways_image_df.head()

,identifier_info,coordinates,GPSTimeStamp,GPSDateStamp,drive,mode,route,type,mile_marker,image
0,"[F:, pathways, 0006NR23203, I, 0_0.237, 0.003_...","(39.68843, -110.85625)","(18.0, 34.0, 38.571)",2024:08:06,F:,pathways,0006NR23203,I,0_0.237,0.003_Roadway.jpg
1,"[F:, pathways, 0006NR23203, I, 0_0.237, 0.009_...","(39.6885, -110.85625)","(18.0, 34.0, 38.971)",2024:08:06,F:,pathways,0006NR23203,I,0_0.237,0.009_Roadway.jpg
2,"[F:, pathways, 0006NR23203, I, 0_0.237, 0.014_...","(39.68858, -110.85624)","(18.0, 34.0, 39.371)",2024:08:06,F:,pathways,0006NR23203,I,0_0.237,0.014_Roadway.jpg
3,"[F:, pathways, 0006NR23203, I, 0_0.237, 0.019_...","(39.68865, -110.85624)","(18.0, 34.0, 39.771)",2024:08:06,F:,pathways,0006NR23203,I,0_0.237,0.019_Roadway.jpg
4,"[F:, pathways, 0006NR23203, I, 0_0.237, 0.024_...","(39.68873, -110.85624)","(18.0, 34.0, 40.221)",2024:08:06,F:,pathways,0006NR23203,I,0_0.237,0.024_Roadway.jpg


In [68]:
pathways_image_df.drop(['identifier_info','drive'],axis = 1, inplace = True)

In [69]:
pathways_image_df.to_csv("pathways_data.csv", index = False)

In [20]:
pathways_data = pd.read_csv("pathways_data.csv")

In [10]:
pathways_data.dtypes

coordinates     object
GPSTimeStamp    object
GPSDateStamp    object
mode            object
route           object
type            object
mile_marker     object
image           object
dtype: object

In [13]:
pathways_data.to_csv("pathways_data_updated.csv", index = False)

In [ ]:
import geopandas as gpd

In [ ]:
pathways_data[["latitude_name", "longitude_name"]] = pathways_data["coordinates"].str.strip('()').str.split(', ', expand=True)

    # Convert to numeric (float)
pathways_data["lat"] = pd.to_numeric(pathways_data["latitude_name"], errors='coerce')
pathways_data["lon"] = pd.to_numeric(pathways_data["longitude_name"], errors='coerce')

In [ ]:
gdf = gpd.GeoDataFrame(pathways_data, geometry = gpd.points_from_xy(pathways_data.lon, pathways_data.lat))

In [ ]:
gdf.drop(["coordinates", "latitude_name", "longitude_name"], axis = 1, inplace = True)

In [ ]:
gdf["GPSDateStamp"] = gdf["GPSDateStamp"].str.replace(":", "-")

In [ ]:
gdf.to_csv("pathways_data_geometry.csv", index = False)

In [ ]:
from google.cloud import storage

# Create a storage client
storage_client = storage.Client()

# Specify the bucket and file names
bucket_name = 'udot-pathways-data'
blob_name = 'pathways_data_geometry.csv'

# Upload the file
bucket = storage_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename('pathways_data_geometry.csv')

In [ ]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

# TODO(developer): Set table_id to the ID of the table to create.
table_id = "ut-udot-adap-prod.pathways.image_metadata"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("GPSTimeStamp", "STRING"),
        bigquery.SchemaField("GPSDateStamp", "DATE"),
        bigquery.SchemaField("mode", "STRING"),
        bigquery.SchemaField("route", "STRING"),
        bigquery.SchemaField("type", "STRING"),
        bigquery.SchemaField("mile_marker", "STRING"),
        bigquery.SchemaField("image", "STRING"),
        bigquery.SchemaField("lat", "NUMERIC"),
        bigquery.SchemaField("lon", "NUMERIC"),
        bigquery.SchemaField("geometry", "GEOGRAPHY"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
    write_disposition = "WRITE_TRUNCATE",
    
)
uri = "https://storage.cloud.google.com/udot-pathways-data/pathways_data_geometry.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Wait for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 1945840 rows.
